In [138]:
%config IPCompleter.greedy=True
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
from receipt_detector import is_receipt
from datetime import datetime,timedelta
from dateutil.parser import parse
import pytz


In [139]:
# If modifying these scopes, delete the file token.json.
SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'

In [140]:
store = file.Storage('token.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('gmail', 'v1', http=creds.authorize(Http()))


In [141]:
user_id =  'me'
label_id_one = 'INBOX'
label_id_two = 'UNREAD'
    
def get_messages(date_query):
    messages = []
    try:
        response = service.users().messages().list(userId=user_id,labelIds=[label_id_one],q=date_query).execute()
        
        if 'messages' in response:
          messages.extend(response['messages'])

        while 'nextPageToken' in response:
          page_token = response['nextPageToken']
          response = service.users().messages().list(userId=user_id,labelIds=[label_id_one],q=date_query,
                                             pageToken=page_token).execute()
          messages.extend(response['messages'])
          
    except error:
        print ('An error occurred: %s' % error)
    return messages

In [142]:
def check_for_receipt(message):
    payld = message['payload'] # get payload of the message 
    headr = payld['headers'] # get header of the payload

    for one in headr: # getting the Subject
        if one['name'] == 'Subject':
            msg_subject = one['value']
            if is_receipt(msg_subject):
                return True,msg_subject
    return False,None

    

In [146]:
import time

def on_complete(request_id, response, exception):
    if exception is not None:
        print(exception)
    else:
        got_receipt, subject = check_for_receipt(response)
        if got_receipt:
            print(subject)


messages = get_messages('after:2018/01/13 before:2018/02/13')
print(len(messages))


chunk_size = 50

start = 0
while start <= len(messages):

    batch = service.new_batch_http_request()
    end = start + chunk_size
    
    if end >= len(messages):
        end = len(messages) - 1

    for i in range(start,end):
        batch.add(service.users().messages().get(userId=user_id, id=messages[i]['id']), callback=on_complete)

    batch.execute()
    start += chunk_size
    

511
Monthly Parking Invoice
Receipt for Your Payment to GitHub, Inc.
[GitHub] Payment Receipt for thisislance98
Your Corporate Card Statement is Ready
Now online: Your health statement from UnitedHealthcare
Receipt: Subscription payment #12641867
We've received your payment
We've received your payment
Payment Reveipt
Confirmation of Your Bank Enrollment
"Canada's national anthem is now gender neutral 🇨🇦" Moment
Confirmation from Farmers Insurance: Enrollment in automatic payments
Regarding your Electronic Payment Privileges
[Revmob] Paypal Payment Processed
Revmob payment
Ski → Stay
Receipt for Your Payment to iTunes and App Store
Confirmation of Your Bank Enrollment
We've received your payment
Payment Confirmation
Spelling Bee 2018
LAST DAY TO SAVE: Take $25 OFF Every Order Over $250 OR Save $200 on Super Bowl Tickets!
Billing agreement cancellation confirmation
CES 2018 Highlights
Winter Sale: Take $25 OFF Every Order Over $250 OR Save $200 on Super Bowl Tickets!
"Florida man tries t